<a href="https://colab.research.google.com/github/SriSatyaLokesh/some-app-for-whatsapp/blob/master/notebooks/analytics/Visualization_Vasavi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving WhatsApp Chat with Priyanka_VL.txt to WhatsApp Chat with Priyanka_VL.txt


# DataFrame

In [11]:
import pandas as pd
from itertools import chain, repeat
import re

chainer = chain.from_iterable
class Cleaner:
  def __init__(self,filename):
    self.filename = filename

  def _clean_data(self):
    chat_file = self.filename
    with open(chat_file) as file:
      all_msgs = file.readlines()
      raw_texts = []
      for line in all_msgs:
        if re.match("^\d{2}/\d{2}/\d{2,4},\s\d{1,2}:\d{2}\s[ap]*[m]*",line):
          date_split = line.split(' - ') #date and user
          user_split = date_split[1].split(':')
          if len(user_split) >= 2:
            raw_texts.append(line)
        else:
          try:
            raw_texts[-1] += line
          except Exception as err:
            # print(line,raw_texts)
            # print(err)
            pass

    return raw_texts 
      
  def _get_data(self):
    raw_texts = self._clean_data()
    raw_messages = dict()
    for line in raw_texts:
      date_split = line.split(' - ')
      user_split = date_split[1].split(':')
      user = user_split[0]
      if user not in raw_messages:
        raw_messages[user] = [line]
      else:
        raw_messages[user].append(line)

    df = pd.DataFrame({'user': list(chainer(repeat(k, len(v)) for k,v in raw_messages.items())),
                   'raw_text': list(chainer(raw_messages.values()))}) 
    return df

chat_file = "WhatsApp Chat with CSE 2K16-2K20_VASAVI.txt"
clean = Cleaner(chat_file)
df = clean._get_data()
df

,user,raw_text
0,+91 73311 74849,"17/07/19, 8:21 pm - +91 73311 74849: Yes sir\n"
1,+91 73311 74849,"02/08/19, 7:23 pm - +91 73311 74849: <Media om..."
2,+91 80965 40660,"17/07/19, 8:32 pm - +91 80965 40660: Sir tomm ..."
3,+91 80965 40660,"08/08/19, 6:28 pm - +91 80965 40660: This mess..."
4,+91 98667 58870,"17/07/19, 8:32 pm - +91 98667 58870: Infy TQ\n"
...,...,...
632,+91 98483 13908,"25/04/20, 1:30 pm - +91 98483 13908: This mess..."
633,+91 98483 13908,"25/04/20, 1:33 pm - +91 98483 13908: Sir, most..."
634,+91 79934 07020,"25/04/20, 1:40 pm - +91 79934 07020: Sir, plea..."
635,+91 73828 52692,"25/04/20, 1:41 pm - +91 73828 52692: Sir, plea..."


In [12]:
from string import digits
df["raw_message"] = df.apply(lambda df : "".join(df['raw_text'].split(":")[2:]),axis=1)
df["raw_message"] = df.apply(lambda df : df['raw_message'][:-1],axis=1)
remove_digits = str.maketrans('', '', digits)
df["text_only_message"] = df.apply(lambda df : df["raw_message"].encode('ascii', 'ignore').decode('ascii'),axis=1)
df["text_only_message"] = df.apply(lambda df : df["text_only_message"].translate(remove_digits),axis=1)
df.loc[(df['text_only_message'] == " This message was deleted") | (df['text_only_message'] == " <Media omitted>"),'text_only_message'] = ""
df = df.groupby('user').filter(lambda x : len(x) > 10)

df['date'] =  df.apply(lambda df : df['raw_text'].split(" - ")[0],axis=1)

temp = ["%d/%m/%Y, %I:%M %p" , "%d/%m/%y, %I:%M %p" , "%d/%m/%Y, %H:%M" , "%d/%m/%y, %H:%M" ,
        "%d/%Y/%m, %I:%M %p" , "%d/%y/%m, %I:%M %p" , "%d/%Y/%m, %H:%M" , "%d/%y/%m, %H:%M" ,
        "%Y/%m/%d, %I:%M %p" , "%y/%m/%d, %I:%M %p" , "%Y/%m/%d, %H:%M" , "%y/%m/%d, %H:%M" ,
        "%Y/%d/%m, %I:%M %p" , "%y/%d/%m, %I:%M %p" , "%Y/%d/%m, %H:%M" , "%y/%d/%m, %H:%M" ,
        "%m/%Y/%d, %I:%M %p" , "%m/%y/%d, %I:%M %p" , "%m/%Y/%d, %H:%M" , "%m/%y/%d, %H:%M" ,
        "%m/%d/%Y, %I:%M %p" , "%m/%d/%y, %I:%M %p" , "%m/%d/%Y, %H:%M" , "%m/%d/%y, %H:%M"]

for formats in temp:       
  try:
    df['date'] = pd.to_datetime(df['date'], format=formats)
  except:
    continue

df['hour'] = df['date'].dt.hour
df['weekday'] = df['date'].dt.weekday
df

,user,raw_text,raw_message,text_only_message,date,hour,weekday
4,+91 98667 58870,"17/07/19, 8:32 pm - +91 98667 58870: Infy TQ\n",Infy TQ,Infy TQ,2019-07-17 20:32:00,20,2
5,+91 98667 58870,"17/07/19, 8:36 pm - +91 98667 58870: Ok\n",Ok,Ok,2019-07-17 20:36:00,20,2
6,+91 98667 58870,"25/07/19, 8:28 pm - +91 98667 58870: satya how...",satya how many from vardhaman,satya how many from vardhaman,2019-07-25 20:28:00,20,3
7,+91 98667 58870,"25/07/19, 8:28 pm - +91 98667 58870: thanks\n",thanks,thanks,2019-07-25 20:28:00,20,3
8,+91 98667 58870,"25/07/19, 8:29 pm - +91 98667 58870: cvr\n",cvr,cvr,2019-07-25 20:29:00,20,3
...,...,...,...,...,...,...,...
620,+91 99126 48686,"15/04/20, 10:19 am - +91 99126 48686: Guys las...",Guys last 2 minutes,Guys last minutes,2020-04-15 10:19:00,10,2
621,+91 99126 48686,"15/04/20, 10:19 am - +91 99126 48686: Exam wil...",Exam will closes,Exam will closes,2020-04-15 10:19:00,10,2
622,+91 99126 48686,"15/04/20, 10:30 am - +91 99126 48686: <Media o...",<Media omitted>,,2020-04-15 10:30:00,10,2
623,+91 99126 48686,"15/04/20, 10:38 am - +91 99126 48686: <Media o...",<Media omitted>,,2020-04-15 10:38:00,10,2


# Weekday Graph

In [0]:
import plotly.express as px

In [14]:
weekday_graph = pd.DataFrame({'weekday' : ['mon','tue','web','thr','fri','sat','sun'],'count' : df.groupby('weekday').size()})
weekday_graph = weekday_graph.reset_index(drop = True)
weekday_graph

,weekday,count
0,mon,55
1,tue,78
2,web,70
3,thr,105
4,fri,49
5,sat,50
6,sun,32


In [15]:
fig = px.line_polar(weekday_graph, r="count", theta="weekday", line_close=True)
fig.show()

# Each user message count graph

In [16]:
user_count =  pd.DataFrame({'msg_count': df.groupby('user').size()})
user_count = user_count.reset_index(drop = False)
user_count

,user,msg_count
0,+91 88013 66964,12
1,+91 97044 13836,17
2,+91 98667 58870,91
3,+91 99126 48686,31
4,+91 99789 67155,33
5,Hemanth_VMEG,23
6,IDIOT😎,19
7,Varma Sir_PAT,213


In [17]:
fig1 = px.bar_polar(user_count, r="msg_count", theta="user", template="plotly_dark")
fig1.update_polars(angularaxis_type="category")
fig1.show()